# Foreword

We welcome you to use rLLMFT, a sub-implementation of the PCRLLM research, to replicate the experiments in the paper *PCRLLM: Proof-Carrying Reasoning with Large Language Models under Stepwise Logical Constraints*. Since we don't have a powerful GPU, we can only present this as a Colab script. Below is an introduction to using this script:

1) Data Generation: We will generate training data **for different LLMs** and test data for all models all at once.

2) Each *cycle* involves fine-tuning and testing a specific LLM, generating a file for logging after testing. This cycle should be performed once for each LLM. However, since this is a Colab script, completely uninstalling a model while maintaining a session is somewhat complex. Therefore, we need you to repeatedly disconnect/connect to this script as you change the *model index* to **download test data**.

3) Final Scoring: After completing the last cycle, you can use the grading box. You will need to upload the test data files for all previous models.

Thank you for using this script! If you have any questions, feel free to send email to tuo90515@temple.edu.

# Config & Datagen

In [ ]:
TOKENIZER_LENGTH = 800

NUM_FINETUNE_DATA = 100
NUM_FINETUNE_EPOCHS = 2

# model_name = "Qwen/Qwen2.5-0.5B"
model_name = "Qwen/Qwen2.5-1.5B"

NUM_MODELS = 3

USER_TAG = "User:"
ASSISTANT_TAG = "Assistant:"

seed = 39

In [ ]:
!git clone https://github.com/MoonWalker1997/rLLMFT.git
!python ./rLLMFT/data_gen.py --num_data {NUM_FINETUNE_DATA} --num_models {NUM_MODELS} --random_seed {seed}

In [ ]:
import gc
import csv

import pandas as pd
import torch
import tqdm
from torch.utils.data import DataLoader

import transformers
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments)

## Util

In [ ]:
def get_full_text(row):
    intro = row["Introduction"].strip()
    premise = row["Premise"].strip()
    question = row["Question"].strip()
    answer = row["Answer"].strip()
    return f"{intro}\n{USER_TAG} {premise} {question}\n{ASSISTANT_TAG} {answer}{tokenizer.eos_token}"


def preprocess(row):
    full_text = get_full_text(row)
    asst_idx = full_text.rfind(ASSISTANT_TAG)
    enc = tokenizer(full_text, truncation=True, max_length=TOKENIZER_LENGTH)
    input_ids = enc["input_ids"]

    prefix_text = full_text[:asst_idx]
    prefix_ids = tokenizer(prefix_text, truncation=True, max_length=TOKENIZER_LENGTH)["input_ids"]
    mask_start = len(prefix_ids)

    labels = [-100] * mask_start + input_ids[mask_start:]
    enc["labels"] = labels
    return enc

def get_full_text_test(row):
    intro = row["Introduction"].strip()
    premise = row["Premise"].strip()
    question = row["Question"].strip()
    return f"{intro}\n{USER_TAG} {premise} {question}\n{ASSISTANT_TAG}"

def preprocess_for_generate(row):
    full_text = get_full_text_test(row)
    return tokenizer(
        full_text,
        truncation=True,
        max_length=TOKENIZER_LENGTH,)

# Cycle

In [ ]:
MODEL_INDEX = 2

## Full-Finetune

### Load Data and Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = TOKENIZER_LENGTH
model = AutoModelForCausalLM.from_pretrained(model_name)

df = pd.read_csv(f"./data/data_table_{MODEL_INDEX}_{NUM_MODELS}.csv",
                 quotechar='"',
                 doublequote=True)
dataset = Dataset.from_pandas(df)

### Training

In [ ]:
tokenized_dataset = dataset.map(
    preprocess,
    batched=False,
    remove_columns=dataset.column_names)

training_args = TrainingArguments(
    output_dir="./finetuned_model",
    per_device_train_batch_size=1,
    num_train_epochs=NUM_FINETUNE_EPOCHS,
    logging_steps=10,
    save_strategy="no",
    bf16=True,
    gradient_accumulation_steps=4,
    report_to=[])

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset)

print("Start...")
trainer.train()
print("Finish")

## Testing

In [ ]:
df_test = pd.read_csv("./data/data_table_test.csv",
                      quotechar='"',
                      doublequote=True)

dataset_test = Dataset.from_pandas(df_test)

tokenized_test = dataset_test.map(
    preprocess_for_generate,
    batched=False,
    remove_columns=dataset_test.column_names)

dataloader = DataLoader(
    tokenized_test,
    batch_size=16,
    shuffle=False,
    collate_fn=lambda batch: {
        "input_ids": torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(x["input_ids"]) for x in batch],
            batch_first=True,
            padding_value=tokenizer.pad_token_id
        ),
        "attention_mask": torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(x["attention_mask"]) for x in batch],
            batch_first=True,
            padding_value=0
        )
    })

model.eval()
model.to("cuda")

transformers.logging.set_verbosity_error()

outputs = []
with torch.no_grad():
    for batch in tqdm.tqdm(dataloader):
        batch = {k: v.to("cuda") for k, v in batch.items()}
        generated = model.generate(
            **batch,
            max_new_tokens=TOKENIZER_LENGTH - batch["input_ids"].shape[1],
            num_beams=1,
            do_sample=False
        )
        texts = tokenizer.batch_decode(generated, skip_special_tokens=True)
        outputs.extend(texts)

tmp = []
for i in range(len(dataset_test)):
    tmp.append([get_full_text(dataset_test[i]), outputs[i]])

with open(f"./test_record/test_record_{MODEL_INDEX}_{NUM_MODELS}.csv", "w") as f:
    writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Label", "Output"])
    for row in tmp:
        writer.writerow(row)

# Grading

In [ ]:
import shutil

from google.colab import files
uploaded = files.upload()

import os

for old_name in uploaded.keys():
    new_name = old_name.split("-")[0] + ".csv"
    os.rename(old_name, new_name)

for i in range(NUM_MODELS):
    try:
        shutil.move(f"./test_record_{i}_{NUM_MODELS}.csv", f"./test_record/test_record_{i}_{NUM_MODELS}.csv")
    except:
        continue

!python ./rLLMFT/grading.py